<a href="https://colab.research.google.com/github/belyakov23/intro-ml-projects-/blob/main/cats_dogs_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cats vs Dogs Image Classification

This project uses the Kaggle "Cats and Dogs image classification" dataset to train a convolutional neural network that distinguishes between cat and dog images. The data consists of JPEG images organised into train and test folders with separate subfolders for cats and dogs.

I preprocess the images with Keras `ImageDataGenerator`, rescaling pixels and creating training, validation, and test generators. Then I build a small CNN with three convolution–max pooling blocks followed by dense layers and train it with binary cross-entropy loss and the Adam optimizer.

The final model reaches about 89% training accuracy and around 67% validation accuracy after 10 epochs, with test accuracy close to the validation score. This shows a basic but functional image classifier on a real-world, slightly noisy cats-and-dogs dataset.



In [10]:
!pip install -q kagglehub

import kagglehub
import os

# Download latest version of the dataset
path = kagglehub.dataset_download("samuelcortinhas/cats-and-dogs-image-classification")

print("Path to dataset files:", path)
print(os.listdir(path))


Using Colab cache for faster access to the 'cats-and-dogs-image-classification' dataset.
Path to dataset files: /kaggle/input/cats-and-dogs-image-classification
['test', 'train']


In [11]:
for root, dirs, files in os.walk("/content/cats_dogs_data"):
    print(root, "->", len(files), "files")


/content/cats_dogs_data -> 0 files


In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


In [13]:
train_dir = os.path.join(path, "train")
test_dir = os.path.join(path, "test")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

img_size = (150, 150)
batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="validation"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False
)


Found 447 images belonging to 2 classes.
Found 110 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [14]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.5036 - loss: 1.0948 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.6006 - loss: 0.6926 - val_accuracy: 0.5000 - val_loss: 0.6938
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - accuracy: 0.5232 - loss: 0.6925 - val_accuracy: 0.4909 - val_loss: 0.6933
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.5411 - loss: 0.6863 - val_accuracy: 0.5182 - val_loss: 0.7032
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.6053 - loss: 0.6500 - val_accuracy: 0.5727 - val_loss: 0.6869
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.5749 - loss: 0.6284 - val_accuracy: 0.6182 - val_loss: 0.6282
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.7136 - loss: 0.5324 - val_accuracy: 0.6000 - val_loss: 0.6645
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.8043 - loss: 0.4265 - val_accuracy: 0.6545 - val_loss:

In [16]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 431ms/step - accuracy: 0.7023 - loss: 0.6707
Test accuracy: 0.6642857193946838


In [17]:
import numpy as np

test_generator.reset()
imgs, labels = next(test_generator)

preds = model.predict(imgs)
pred_classes = (preds.flatten() > 0.5).astype(int)

print("True labels:     ", labels[:10].astype(int))
print("Predicted labels:", pred_classes[:10])
print("Class indices:", test_generator.class_indices)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step
True labels:      [0 0 0 0 0 0 0 0 0 0]
Predicted labels: [1 0 0 0 0 0 0 0 0 1]
Class indices: {'cats': 0, 'dogs': 1}
